In [21]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [22]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'rated_capacity_amperage'

In [23]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [24]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
field='RatedCapacity'
fields = [field]
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

919


In [25]:
# df[field].explode().value_counts()
df.head(0)

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, RatedCapacity]
Index: []

In [26]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [27]:
# df['MaxPressure'].explode().value_counts()

In [28]:
# start=r'(?i)(^.{0,10})|()'
# df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(?i)(\d+\s?(?:Amp))|()''' 
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))



pat=r'(?i)(\d+\s?(?:A\W|amp))|()'
df['matches']=df[field].apply(lambda x: re_extract(pat, str(x)))

# pat=r'(?i)(m)|()'
# df['mm']=df[field].apply(lambda x: re_extract(pat, str(x)))


custom=df[(df[field].astype(str)!='nan')]
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x))).apply(lambda x: re.sub(r'(?i)\s?mm',' mm',str(x))).apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x))).apply(lambda x: re.sub(r'\.0','',str(x))).apply(lambda x: re.sub(r'(?i)\s?Am?p?s?','A',str(x)))#.apply(lambda x: re.sub(r'(?i)\s?mm\s?',' mm',str(x))).apply(lambda x: re.sub(r'\s?"\s?',' in',str(x))).apply(lambda x: f'{x} in').apply(lambda x: re.sub(r'\s?in\s?in',' in',str(x)))                                          
# custom[curation_col]=custom[field].apply(lambda x: f'{x}"').apply(lambda x: re.sub(r'""','"',str(x)))
print(len(custom))
matchcustom=custom[['external_id',curation_col]]
matchcustom[curation_col]=matchcustom[curation_col]
matchcustom=matchcustom[matchcustom[curation_col].astype(str)!='']
matchcustom[curation_col].explode().value_counts()

151


20A    34
30A    26
14A    11
46A    10
42A     9
Name: Q:rated_capacity_amperage, dtype: int64

In [29]:
not_match=df[(df[field].astype(str)=='nan')]
# print(len(not_match[(not_match['ld'].astype(str)=='[]')|(not_match['name'].astype(str)!='[]')]))
two=not_match[(not_match['ld'].astype(str)!='[]')|(not_match['name'].astype(str)!='[]')]
# print(len(two))
# two[curation_col]=two['name'].apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x))).apply(lambda x: re.sub(r"'",'"',str(x)))


two[curation_col]=two['ld'].apply(lambda x: re.sub(r'(?i)\s?amp','A',str(x)))
matchtwo=two[['external_id',curation_col]]
print(len(matchtwo))
two

37


buckets external_id  \
1    Booster/Jumper Cables       26837   
2    Booster/Jumper Cables       26838   
3    Booster/Jumper Cables       26836   
4    Booster/Jumper Cables       26835   
69       Electrical System      271333   
70       Electrical System      271334   
71       Electrical System      271335   
72       Electrical System      271336   
73       Electrical System      271337   
74       Electrical System      271338   
506      Electrical System       22105   
510      Electrical System       22106   
556      Electrical System       22109   
643      Electrical System      221020   
653      Electrical System       22747   
654      Electrical System       22753   
656      Electrical System       22749   
657      Electrical System       22755   
658      Electrical System       22750   
660      Electrical System       22748   
663      Electrical System       22752   
665      Electrical System       22756   
666      Electrical System       22751   
668      Electrical System       22754   
669      Electrical System       22757   
670      Electrical System       22758   
689      Electrical System      221023   
690      Electrical System      221017   
691      Electrical System      221021   
692      Electrical System      221015   
693      Electrical System      221022   
694      Electrical System      221016   
695      Electrical System      221012   
696      Electrical System      221018   
697      Electrical System      221014   
698      Electrical System      221019   
699      Electrical System      221013   

                                        product_name  \
1             2/0 AWG Extreme Duty Booster Cable 20'   
2             2/0 AWG Extreme Duty Booster Cable 30'   
3                    4 AWG Battery Booster Cable 16'   
4                    4 AWG Battery Booster Cable 12'   
69              50 Amp Battery Connector Safety Boot   
70             175 Amp Battery Connector Safety Boot   
71             350 Amp Battery Connector Safety Boot   
72           50 Amp Protective Battery Connector Cap   
73          175 Amp Protective Battery Connector Cap   
74          350 Amp Protective Battery Connector Cap   
506  General Motors Metripack Single Wire Cable Seal   
510  General Motors Metripack Single Wire Cable Seal   
556  General Motors Metripack Single Wire Cable Seal   
643                 Deutsch DTM Series Locking Wedge   
653                  Deutsch DT Series Locking Wedge   
654                  Deutsch DT Series Locking Wedge   
656                  Deutsch DT Series Locking Wedge   
657                  Deutsch DT Series Locking Wedge   
658                  Deutsch DT Series Locking Wedge   
660                  Deutsch DT Series Locking Wedge   
663                  Deutsch DT Series Locking Wedge   
665                  Deutsch DT Series Locking Wedge   
666                  Deutsch DT Series Locking Wedge   
668                  Deutsch DT Series Locking Wedge   
669                  Deutsch DT Series Locking Wedge   
670                  Deutsch DT Series Locking Wedge   
689                 Deutsch DTM Series Locking Wedge   
690                 Deutsch DTM Series Locking Wedge   
691                 Deutsch DTM Series Locking Wedge   
692                 Deutsch DTM Series Locking Wedge   
693                 Deutsch DTM Series Locking Wedge   
694                 Deutsch DTM Series Locking Wedge   
695                 Deutsch DTM Series Locking Wedge   
696                 Deutsch DTM Series Locking Wedge   
697                 Deutsch DTM Series Locking Wedge   
698                 Deutsch DTM Series Locking Wedge   
699                 Deutsch DTM Series Locking Wedge   

                                                                                                                                                                                                                                                                                                     

In [30]:
# df[(df['name'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')][0:500]

In [31]:
# two=df[(df['matches'].astype(str)=='[]')]
# print(len(two[two['name'].astype(str)!='[]']))
# two=two[two['name'].astype(str)!='[]']
# two[curation_col]=two['name'].apply(lambda x:re.sub(r'(?i)P\.?S\.?I\.?','',str(x))).apply(lambda x: re.sub(r'.+\-\s?',"['",str(x))).apply(lambda x: re.sub(r'(?<!\,)000',',000',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"\s?'\]",'"]',str(x))).apply(lambda x: re.sub(r"\s?\s?\s?'\s?\,\s?'",'","',str(x))).apply(lambda x: re.sub(r'200"\,"125','125","200',str(x))).apply(lambda x: re.sub(r'\["1\,500"\,"100"\,"2,000 "\]','["100","1,500","2,000"]',str(x))).apply(lambda x: re.sub(r'\s?"\]','"]',str(x)))                                                    
# two[curation_col].explode().value_counts()
# matchtwo=two[['external_id',curation_col]]

In [32]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [33]:
na=df[(df['name'].astype(str)=='[]')&(df[field].astype(str)=='nan')&(df['ld'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
# print(len(na)+len(matchcustom)+len(matchtwo))
na

731


buckets external_id  \
0    Booster/Jumper Cables       26839   
5        Electrical System       40233   
6        Electrical System      402333   
12       Electrical System      402396   
13       Electrical System      402701   
..                     ...         ...   
914      Electrical System       26875   
915      Electrical System       26872   
916      Electrical System    271204RE   
917      Electrical System    271209RE   
918      Electrical System      279991   

                                                                                 product_name  \
0                                                                Quick Connect Booster Cables   
5                                                  Weatherpack Automotive Terminal Assortment   
6                                                          General Motors Terminal Assortment   
12                                                              Heavy Duty Pigtail Assortment   
13                                                    Lead Plated Battery Terminal Assortment   
..                                                                                        ...   
914                                                  8 mm 90 Degree Neoprene Distributor Boot   
915                                                  8 mm Ford 90 Degree Neoprene Coil Nipple   
916        Refill Kit for Kim-Tech Twist-On Battery Terminal Service Truck / Field Repair Kit   
917  Refill Kit for 4 AWG Kim-Tech Twist-On Battery Terminal Service Truck / Field Repair Kit   
918                                                             6 AWG Solderconn Battery Slug   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   long_desc  \
0                                                                                                                                                                                                                                                                                                                                                    HIGH AMP QUICK CONNECT PLUGS - Allows fast connection between the charging vehicle and the vehicle being jumped. Includes a plug cover to prevent damage to the charging vehicle connector when not in use. 5-FOOT VEHICLE SIDE CABLES - Allows easy access to battery power without opening the hood of the charging vehicle. 25-FOOT VEHICLE JUMPER CABLES - Allows easy reach to the vehicle being jumped. Battery clamps securely grip side terminals, post terminals and remote battery access terminals. HEAVY DUTY CONSTRUCTION - Two gauge stranded copper conductor with a tough PVC outer cover provides long service life. MADE IN THE U.S.A   
5                                                                                                                                                                                                                                                                                                                                                                                                            

In [18]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchtwo))

# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [11]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [19]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [20]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [30]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchtwo) 